# Методы первой необходмости

In [2]:
import os
os.environ['HF_HOME'] = 'D:\\HuggingFace'
os.environ['TRANSFORMERS_CACHE'] = os.environ['HF_HOME']
os.environ['HUGGINGFACE_HUB_CACHE'] = os.environ['HF_HOME'] 

In [13]:
from warnings import filterwarnings
filterwarnings('ignore')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import pipeline
import re

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sns.set_style('darkgrid')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Предобработка текстов в pdf и создание датасета

In [11]:
import PyPDF2

In [12]:
clean_data = pd.DataFrame({"Informatics": [], "Art": [], "Space": [], "Sport": [], "Chemical": []})

In [166]:
import pathlib

#books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Образование")
#books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Спорт")
#books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Космос")
#books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Химия")
books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Искусствоведение")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

In [167]:
files_to_process

['D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\1.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\10.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\11.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\12.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\13.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\14.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\15.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\16.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\17.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\18.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\Electronic_library\\dataset\\Искусствоведение\\19.pdf',
 'D:\\ПРОГА\\Проектики\\Github\\E

In [103]:
with open(files_to_process[0], "rb") as f:
    reader = PyPDF2.PdfReader(f)
    text = [page.extract_text() for page in reader.pages]

In [174]:
def process_info(file):
    with open(file, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ''.join([page.extract_text() for page in reader.pages])
    # Служебная информация
    text = re.sub(r'ISSN\s+\d{4}-\d{3,4}[^\n]*', '', text)
    text = re.sub(r'\d{4};\d{2}\(\d+\):\d+–\d+', '', text)

    # Авторы
    text = re.sub(r'[А-ЯЁ][а-яё]+\s+[А-ЯЁ][\.\s]+\s*[А-ЯЁ][\.\s]*', '', text)
    text = re.sub(r'[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][\.\s]+\s*[А-ЯЁ][\.\s]*', '', text)
    text = re.sub(r'\d+[\s\w\.,–-]+(университет|институт|академия|центр)[^\n]*', '', text)

    # Сноски в квадратных скобках
    text = re.sub(r'\[\d+\]', '', text)  # [1], [2]
    text = re.sub(r'\[\d+[,-]\d+\]', '', text)  # [1-3], [4,5]
    text = re.sub(r'\[[A-Za-z]+\d*\]', '', text)  # [A1], [B]
    
    # email
    text = re.sub(r'\S+@\S+', '', text)
    
    # английские разделы 
    text = re.sub(r'Abstract[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    text = re.sub(r'Keywords[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    text = re.sub(r'For citation[^\n]*[\s\S]*?(?=\n[А-ЯЁ]|$)', '', text)
    
    # ссылки
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'DOI:\s*\S+', '', text)
    
    # библиография
    text = re.sub(r'Список\s+источников[\s\S]*', '', text)
    text = re.sub(r'References[\s\S]*', '', text)
    
    # спец.символы
    text = text.replace('\xa0', ' ').replace('•', '')
    text = re.sub(r'-\s+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'•\s*\n', '', text)
    
    # оставшиеся английские фрагменты
    text = re.sub(r'(?:[A-Za-z-]+\s){3,}[A-Za-z-]*', '', text)
    
    return text

## Тексты образования

In [175]:
books = pathlib.Path("D:/ПРОГА/Проектики/Github/Electronic_library/dataset/Образование")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

clean_files = []
for file in files_to_process:
    clean_files.append(process_info(file))

clean_data['Informatics'] = clean_files

In [177]:
clean_data['Informatics'][1]

"ИНФОРМАТИЗАЦИЯ ОБРАЗОВАНИЯ / г. Москва, Россия 3 «Инскултех», г. Москва, Россия 4 Лицей № АннотацияПроблема недостаточного когнитивного развития школьников может быть решена через организацию персонализированной внеучебной деятельности с помощью рекомендательного сервиса, созданного с опорой на нейронаучные исследования и современные знания о работе мозга. Основой для составления персонализированного списка рекомендуемых видов внеучебной деятельности становятся данные, полученные с помощью технологии нейровизуализации в процессе тестирования когнитивных и личностных характеристик учащегося, вкупе с информацией о его интересах и предпочтениях. Регулярное тестирование, проводимое каждые три—шесть месяцев, позволяет определить эффективность выбранных видов деятельности и скорректировать образовательный маршрут. Предполагается, что регулярное использование рекомендательного сервиса позволит стимулировать развитие когнитивных функций ребенка, тем самым способствуя повышению его академическ

In [113]:
process_info(files_to_process[0])

Ученые записки университета имени П.Ф. Лесгафта. 2025. № 4 (242) ФИЗИЧЕСКАЯ КУЛЬТУРА И ПРОФЕССИОНАЛЬНАЯ ФИЗИЧЕСКАЯ ПОДГОТОВКА УДК 796.011.3 DOI 10.5930/ Аннотация Цель исследования разработка и обоснование эффективности педагогического про­ екта «В будущее со спортом!» с применением инновационных фиджитал-технологий. Методы и организация исследования. Использовали теоретические методы (анализ научно-методической литературы, проектирование результатов и процессов их достижения); диагностические (анкетирование, опрос, офлайн и онлайн регистрация); математическая и статистическая обработка данных. Разработанный проект был апробирован в процессе физ­ культурно-спортивной работы со студентами очной формы обучения Кубанского государ­ ственного университета. Результаты исследования и выводы. Анализ полученных результатов доказал эф­ фективность разработанного проекта «В будущее со спортом!». Развитие и популяризация технологичных видов спорта и фиджитал-дисциплин в вузе наряду с классическими

In [160]:
process_info(files_to_process[2])

Солнечно -земная физика. 2024. Т. 10. № 4 Solnechno-zemnaya fizika. 202 4. Vol. 10. Iss. 4 22 УДК 551.51, 532.5 Поступила в редакцию 17.06.2024 -104202403 Принята к публикации 22.10.2024 ЗАВИСИМОСТЬ НОРМАЛЬНЫХ МОД БАРОТРОПНОГО УРАВНЕНИЯ ВИХРЯ ОТ СТРУКТУРЫ СРЕДНЕГО ТЕЧЕНИЯ И ПАРАМЕТРОВ ЧИСЛЕННОГО М ОДЕЛИРОВАНИЯ В.И. Мордвинов Институт солнечно -земной физики СО РА Н, Иркутск , Россия, Е.В. Девятова Институт солнечно -земной физики СО РА Н, Иркутск , Россия, devyatova @iszf .irk.ru В.М. Томозов Институт солнечно -земной физики СО РА Н, Иркутск , Россия, V.I. , Irkutsk , Russia, E.V. , Irkutsk , Russia, V.M. , Irkutsk , Russia, Аннотация. В работе представлены резул ьтаты расчетов нормальных мод среднего течения, об условленного суперпозицией циклонического и а нтициклонического вихрей в высоких широтах. П одобная структура потока часто наблюдается з имой в верхней тропосфере — нижней стратосфере. Мы надеялись выделить в спектре колебаний нормал ьные моды, напоминающие крутильные колеб ан

In [165]:
process_info(files_to_process[2])

ХИМИЯ РАСТИТЕЛЬНОГО СЫРЬЯ . 2025. №1. С. 197–207. KHIMIYA RASTITEL 'NOGO SYR 'YA, 2025, no. 1, pp. 197–207. jcprm .20250114925 УДК 615.322:547.913(571) СПОСОБ ИЗВЛЕЧЕНИЯ МАСЛА, ОБОГАЩЕННОГО ДУБИЛЬНЫМИ ВЕЩЕСТВАМИ, ИЗ ШРОТА МОРОШКИ* © В.И. Репина , А.Д. Ивахнов, С.А. Покрышкин, А. Ю. Кожевников** Северный (Арктический) федеральный университет им. М.В. Ломоносова, Центр коллективного пользования научным оборудованием «Арктика», наб. Северной Двины, 17, Архангельск, 163002, Россия, akozhevnikov @mail.ru Морошка – это уникальная ягода, обладающая не только особым вкусом, но и полезными свойствами, в том числе благодаря наличию витаминов группы В, PP и Е и других ценных компонентов. Вследствие этого морошка получила широкое применение в различных отраслях промышленности, в том числе пищевой, где в основном используются ее плоды. Однако семена морошки, которые в настоящее время утилизируются, также содержат в себе большое количество полезных компонентов, наличие которых позволяет их дальнейше

In [172]:
process_info(files_to_process[4])

132 В ТЕАТРАЛЬНЫХ ОПЫТАХ А. ВАСИЛЬЕВА УДК 740 П.В. Куликов Институт философии РАН, сектор эстетики, Москва, Российская Федерация, e-mail: Аннотация. В статье на примере театральных опытов Анатолия Васильева с диалогами Платона выявляется, как философские идеи при переводе с языка литературы и философии на язык театра превращаются в идеи-театр, как их называет Ален Бадью. В 1990-е годы А. Васильев перешел в практике театра Школы драматического искусства от метода психологического (ситуативного) театра к метафизическому театру и игровым структурам. В этом методологическом повороте ре жиссера от театра персонажа к театру персоны, а затем – к театру идей он включает в свою практику диалоги Платона как пропедевтику актерского искусства. В статье проводится театроведческий анализ сценических этюдов Школы драматического искусства по диалогам Платона в сочетании с историкофилософским анализом некоторых ключевых понятий Платоновских диалогов "Г осударство", "Евтидем". Декорации и костюмы в теат